# Contenido 

1. **Carga de librerías y datos**
    - 1.a. Librerías
    - 1.b. Funciones
    - 1.c. Carga de datos
   
2. **Modelado**
    - 2.a. Creación variables exógenas
    - 2.b. Cálculo predictivo

#  1. Carga de librerías y datos

## 1.a. Librerías

In [13]:
import warnings
warnings.filterwarnings("ignore")

# Modelado y Forecasting
# ==============================================================================
import xgboost
import lightgbm
import catboost
import sklearn
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFECV
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.compose import make_column_selector

import skforecast
from skforecast.ForecasterBaseline import ForecasterEquivalentDate
from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.model_selection import bayesian_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.model_selection import select_features
from skforecast.model_selection import backtesting_forecaster
import shap

import cloudpickle
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import yaml

In [14]:
from Creacion_exog import calculo_variables_exogenas

## 1.b. Funciones

In [15]:
def imputar_nulos_por_hora(datos):
    datos.index = pd.to_datetime(datos.index)
    horas = datos.index.hour
    
    # Promedio por hora y sustitución el valores Nan
    media_por_hora = datos.groupby(horas).transform('mean')
    datos = datos.fillna(media_por_hora)
    
    return datos

def auxiliar(variables_exogenas):
    # Selección de variables exógenas a incluir en el modelo
    # ==============================================================================
    exog_cols = []
    # Columnas que terminan con _seno o _coseno son seleccionadas
    exog_cols.extend(variables_exogenas.filter(regex='_seno$|_coseno$').columns.tolist())
    
    # Columnas que empiezan con festivo_ son seleccionadas
    exog_cols.extend(variables_exogenas.filter(regex='^festivo_.*').columns.tolist())
    exog_cols.extend(['FESTIVO'])
    
    variables_exogenas = variables_exogenas.filter(exog_cols, axis=1)
    return exog_cols, variables_exogenas

## 1.c. Carga de datos

In [16]:
# Filtrado de las 498 estaciones para las que se han entrenado modelos
with open("../1-DATOS/3-DATOS DE RESULTADOS/PREDICCION/MODELOS/station_ids.yaml", "r") as file:
    data = yaml.safe_load(file)
stations_ids = data["station_ids"]

In [17]:
df = pd.read_parquet('../1-DATOS/2-DATOS PROCESADOS/BICING/INFORMACION COMPLETA/BICICLETAS_HORARIO_2022_2023_FILTRADO.parquet')
df = df.iloc[:, df.columns.isin(stations_ids)]

# 2. Predicción

## 2.a.  Creación variables exógenas

El conjunto de variables exógenas es independiente de la estación, puesto que no se han estudiado sucesos que afecten únicamente a un grupo de estaciones concretas o una estación específica. Por lo tanto, se genera este conjunto considerando la estación 1 (pero podría haber sido cualquier otra estación).

In [18]:
datos = df[1]
datos = imputar_nulos_por_hora(df)
variables_exogenas = calculo_variables_exogenas(datos)
exog_cols, variables_exogenas = auxiliar(variables_exogenas)

In [19]:
variables_exogenas.head(3)

,mes_seno,mes_coseno,semana_anyo_seno,semana_anyo_coseno,dia_semana_seno,dia_semana_coseno,hora_dia_seno,hora_dia_coseno,hora_amanecer_seno,hora_amanecer_coseno,...,poly_hora_dia_coseno__hora_anochecer_coseno,poly_hora_amanecer_seno__hora_amanecer_coseno,poly_hora_amanecer_seno__hora_anochecer_seno,poly_hora_amanecer_seno__hora_anochecer_coseno,poly_hora_amanecer_coseno__hora_anochecer_seno,poly_hora_amanecer_coseno__hora_anochecer_coseno,poly_hora_anochecer_seno__hora_anochecer_coseno,festivo_dia_anterior,festivo_dia_siguiente,FESTIVO
FECHA,,,,,,,,,,,,,,,,,,,,,
2022-01-01 00:00:00,0.5,0.866025,0.0,1.0,-0.781831,0.62349,0.258819,0.965926,0.866025,-0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1
2022-01-01 01:00:00,0.5,0.866025,0.0,1.0,-0.781831,0.62349,0.500000,0.866025,0.866025,-0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1
2022-01-01 02:00:00,0.5,0.866025,0.0,1.0,-0.781831,0.62349,0.707107,0.707107,0.866025,-0.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1


## 2.b. Cálculo predictivo

Se va a realizar la predicción para el día 5 de Octubre de 2023. Concretamente, para el turno de trabajadores que empieza su jornada a las 08:00h y la termina a las 16:00h.

In [20]:
fecha_inicial_pred = '2023-10-05 09:00:00'
fecha_final_pred = '2023-10-05 16:59:59'

variables_exogenas = variables_exogenas[(variables_exogenas.index >= fecha_inicial_pred)
                                        & (variables_exogenas.index <= fecha_final_pred)]

In [21]:
def prepare_time_series(data, column_name='CANTIDAD', freq='H'):
    data = data.to_frame(name=column_name)
    data.index = pd.to_datetime(data.index)
    data = data.asfreq(freq)
    data.index.name = 'FECHA'
    return data

In [22]:
%%time

fecha_inicial_serie = '2023-10-03 09:00:00'
fecha_final_serie = '2023-10-05 08:59:00'

predicciones_dict = {}

for column in df.columns:

    with open(f'../1-DATOS/3-DATOS DE RESULTADOS/PREDICCION/MODELOS/forecaster_{column}.pkl', 'rb') as f:
        forecaster = cloudpickle.load(f)
    
    data = df[column]
    datos = prepare_time_series(data, column_name='CANTIDAD', freq='H')
    datos = datos[(datos.index >= fecha_inicial_serie) & (datos.index < fecha_final_serie)]
    
    predicciones = forecaster.predict(
        steps=8,
        last_window=datos,
        exog=variables_exogenas
    )
    
    predicciones_dict[column] = predicciones

predicciones_df = pd.DataFrame(predicciones_dict)


CPU times: total: 1min 50s
Wall time: 29.1 s


In [23]:
predicciones_df.head(5)

,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0,...,450.0,452.0,458.0,490.0,465.0,502.0,503.0,52.0,91.0,467.0
2023-10-05 09:00:00,2.156569,2.768908,1.019554,1.193825,5.365381,4.083058,7.145657,15.110778,20.292318,29.578317,...,22.425489,12.361697,11.517286,2.508320,4.013126,0.541409,0.326513,10.068761,1.396575,1.446365
2023-10-05 10:00:00,3.072711,3.649786,1.641659,2.636029,4.495499,5.973305,8.300498,15.147179,19.680962,28.249294,...,21.796550,11.875128,11.203002,2.298003,4.121242,0.587107,0.474877,11.380328,1.492814,0.994589
2023-10-05 11:00:00,3.603284,3.887221,2.840636,4.180569,3.731554,7.267394,9.027734,15.058938,18.584248,26.636195,...,20.704170,12.125994,10.568677,1.755743,5.735199,1.075172,0.798274,11.558055,2.058722,0.920171
2023-10-05 12:00:00,5.606146,4.993168,4.428921,5.134844,3.063963,7.344898,10.741492,14.976689,17.399094,24.685805,...,19.872366,12.543878,9.531323,1.353284,4.072563,1.556492,0.801483,11.275158,3.486854,0.825461
2023-10-05 13:00:00,8.127358,6.214658,5.253170,6.273374,2.894606,5.976444,11.799895,14.896044,16.394078,23.408334,...,19.237659,13.643431,8.948698,1.432675,2.016958,2.060143,1.220908,10.905120,4.903801,0.627060


In [24]:
predicciones_df.to_csv('Predicciones.csv')